In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.action_chains import ActionChains
import urllib
import xlwt
wb = xlwt.Workbook()

country_list = ["CZSS 2019"]
file1= "CZ_stu"

filename = "LQCD_Salary_by_Gender_" + file1

chromeOptions = webdriver.ChromeOptions()
chromeOptions.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome("T:\\Universum Research\\2019\\Training\\Confirmit Export\\chromedriver",chrome_options=chromeOptions, desired_capabilities=chromeOptions.to_capabilities())

driver.maximize_window()
driver.get("https://tabulator.universumglobal.com/ui/default.aspx") 
time.sleep(2)

obj = driver.switch_to.alert
time.sleep(2)
obj.dismiss()

username = driver.find_element_by_id("txtUserName")
password = driver.find_element_by_id("txtPassword")
username.send_keys("pantheon\surendra.kumar")
password.send_keys("Feb@2019")
login_attempt = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[2]/td[2]/table/tbody/tr[8]/td/table/tbody/tr/td/div/a")
login_attempt.submit()


for country_name in country_list:
    try:
        ###survey searching
        time.sleep(5)
        country = country_name
        #x_path = "//td[contains(text(),'" + country + "')]"
        x_path = "//td[text() = '" + country + "']"
        search_survey = driver.find_element_by_xpath(x_path)
        search_survey.click()
        time.sleep(5)

        ########################### FRAMES ##############################
        iframeQuestionGroups = driver.find_element_by_id('iframeQuestionGroups')
        iframeQuestions = driver.find_element_by_id('iframeQuestions')
        iframeAnalyze = driver.find_element_by_id('iframeAnalyze')
        iframeCategorize = driver.find_element_by_id('iframeCategorize')
        iframeAlternatives = driver.find_element_by_id('iframeAlternatives')
        iframeFilter = driver.find_element_by_id('iframeFilter')

        ############################# Questions Group frame###########################
        driver.switch_to.frame(iframeQuestionGroups)
        time.sleep(3)
        if ("Official Target Groups" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Official Target Groups']").click() ##click Official Target Groups            
        elif ("Educational Profile" in driver.page_source):
            driver.find_element_by_xpath("//span[text() = 'Educational Profile']").click()  ##click Educational Profile           
        time.sleep(1)
        driver.switch_to.default_content()

        ############################# Alternatives frame double click to add FILTER ###########################
        actionChains = ActionChains(driver)
        driver.switch_to.frame(iframeAlternatives)
        time.sleep(5)

        #for x3 in ["Young Professionals (", "Senior Professionals (", "Years worked 0 - Professionals ("]:   #  "Young Professionals (", "Senior Professionals (", "Years worked 0 - Professionals ("
        for x3 in ["Students"]:
            x_path2 = '//select/option[contains(text() , "' + x3 + '")]'
            driver.find_element_by_xpath(x_path2).click()
            time.sleep(2)

        time.sleep(5)
        driver.switch_to.default_content()

        ############################# filter frame###########################
        driver.switch_to.frame(iframeFilter)
        time.sleep(3)
        driver.find_element_by_xpath("//a[@href = 'javascript:addFilter(1);']").click()
        driver.switch_to.default_content()

        ############################# Questions frame ###########################
        driver.switch_to.frame(iframeQuestions)
        time.sleep(3)
        select_element = Select(driver.find_element_by_id("listQuestions"))
        select_element.deselect_by_index(0)  # select and unselect Main field of study & official target groups
        #select_element.select_by_index(1)
        select_element.select_by_visible_text("2. Main Field of Study")
        #select_element.select_by_value("Q_0_563317_7_What is your main field of study? / What was your main field of study?")
        #select_element.deselect_by_value("Q_0_563316_3_Official Target Groups")
        driver.switch_to.default_content()

        ############################# Add to Categorize ###########################
        driver.switch_to.frame(iframeCategorize)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listCategorize);']").click()
        driver.switch_to.default_content()

        ############################# Questions Group frame ###########################
        driver.switch_to.frame(iframeQuestionGroups)
        time.sleep(3)
        driver.find_element_by_xpath("//span[text() = 'Career Expectations and Preferences']").click()        
        driver.switch_to.default_content()

        ############################# Questions frame ###########################
        driver.switch_to.frame(iframeQuestions)
        time.sleep(3)
        select_element = Select(driver.find_element_by_id("listQuestions"))
        select_element.deselect_by_index(0)  # select and unselect Main field of study & official target groups
        #select_element.select_by_index(1)
        select_element.select_by_visible_text("[Q] Salary Annual (Local Currency)")        
        driver.switch_to.default_content()   

        ############################# Add to Analyze ###########################
        driver.switch_to.frame(iframeAnalyze)
        time.sleep(3)
        driver.find_element_by_xpath("//a[@onclick ='parent.AddToList(document.forms[0].listAnalyze);']").click()        
        driver.switch_to.default_content()

        ############################# Questions Group frame ###########################
        driver.switch_to.frame(iframeQuestionGroups)
        time.sleep(3)
        driver.find_element_by_xpath("//span[text() = 'Personal Profile']").click()        
        driver.switch_to.default_content()
        time.sleep(5)

        ############################# Questions frame ###########################
        driver.switch_to.frame(iframeQuestions)
        time.sleep(3)
        select_element = Select(driver.find_element_by_id("listQuestions"))
        select_element.deselect_by_index(0)  # select and unselect Gender
        select_element.select_by_visible_text("Gender")
        driver.switch_to.default_content()      
        time.sleep(3)
        ############################## Alternatives frame double click to add FILTER ###########################        
        for gender in ["Female", "Male"]:
            actionChains = ActionChains(driver)
            driver.switch_to.frame(iframeAlternatives)
            time.sleep(3)
            add_filter= driver.find_element_by_xpath("//option[contains(text(), '"+gender+"')]")  ##add student to filter 
            actionChains.double_click(add_filter).perform()
            time.sleep(3)
            driver.switch_to.default_content()

            ############################# Get Table ###########################
            parent_window_handle = driver.current_window_handle
            first_window = driver.window_handles[0]
            driver.find_element_by_xpath("//a[@onclick ='getTable(false)']").click()    
            time.sleep(5)

            ############################ POPUP WINDOWs For data ###########################
            handles = driver.window_handles;
            driver.switch_to.window(handles[1])

            ############################ aply setting HTML format in popup window ###########################
            time.sleep(1)
            #### Output tab
            driver.find_element_by_xpath("//em[text() = 'Output']").click()
            time.sleep(1)
            driver.find_element_by_xpath("//select[@name='listOutput']/option[text()='HTML Table']").click()
            time.sleep(1)
            ### Descriptives tables tab
            driver.find_element_by_xpath("//em[text() = 'Descriptives']").click()
            time.sleep(1)
            driver.find_element_by_xpath("//input[@id = 'chkBoxMedian']").click()  ### Get median 
            driver.find_element_by_xpath("//label[text() ='Minimum']").click()    ####''''''' Get Minimum
            driver.find_element_by_xpath("//label[text() ='Maximum']").click()    ### Get Maximum
            time.sleep(1) 
            #### Total tab
            driver.find_element_by_xpath("//em[text() = 'Total']").click()
            time.sleep(1)
            driver.find_element_by_xpath("//input[@id = 'chkBoxColumnTotal']").click()  ### Click Column Total check box
            time.sleep(1)
            #### Weighting tab
            rowcount = 0
            ws = wb.add_sheet(country + "_" + gender)
            x = 0
            driver.find_element_by_xpath("//em[text() = 'Weighting']").click()  ###
            driver.find_element_by_xpath("//input[@id = 'chkAutomaticWeight']").click()
            for wgt1 in ["No Weight"]:
                driver.switch_to.window(handles[1])
                time.sleep(2)        
                driver.find_element_by_xpath("//select[@name='listWeight']/option[text()='"+wgt1+"']").click()   ### Weighted Total ## Weighted All ##No Weight                    

                ### Apply settings tab
                driver.find_element_by_xpath("//a[text() ='Apply Settings']").click()
                driver.implicitly_wait(30)
                time.sleep(1)

                ###iframe for html data
                iframes_data = driver.find_element_by_xpath("//iframe[contains(@src, 'showData.aspx?guid')]")
                driver.switch_to.frame(iframes_data)

                res = driver.execute_script("return document.documentElement.outerHTML")
                soup = BeautifulSoup(res, 'lxml')

                ################ Put into EXcel##################
                #ws = wb.add_sheet(country+wgt1)
                table = soup.findAll('tbody')[1]  
                rows = table.findAll("tr")
                #x = 0
                for tr in rows:
                    cols = tr.findAll("td")
                    if not cols: 
                        # when we hit an empty row, we should not print anything to the workbook
                        continue
                    y = 0
                    for td in cols:
                        texte_bu = td.text
                        texte_bu = texte_bu.encode('utf-8')
                        texte_bu = texte_bu.strip()
                        if x <= 4:
                            ws.write(x, y, td.text)
                        elif x == 5:    
                            ws.write(x, y + 2, td.text)
                        elif x == 6:    
                            ws.write(x, y, td.text)    
                        else:
                            ws.write(x, y + 1, td.text) 
                        #print(x, y, td.text)
                        y = y + 1
                    # update the row pointer AFTER a row has been printed
                    # this avoids the blank row at the top of your table
                    x = x + 1
                driver.switch_to.window(handles[1])   ### switch to 2nd Html page
            #wb.save('LQCD_Gender.xls')
            ############################# filter frame###########################
            driver.switch_to.window(handles[0])
            driver.switch_to.default_content()
            ###''''''''''''''''''''''''''''''''''''''''''
            driver.switch_to.frame(iframeFilter)
            time.sleep(3)
            select_element2 = Select(driver.find_element_by_id("listFilter"))
            select_element2.select_by_index(1)
            driver.find_element_by_xpath("//a[@href = 'javascript:clearLines();']").click()
            driver.switch_to.default_content()
            ############################# End filter frame###########################
            driver.switch_to.window(handles[0])
            wb.save(filename+'.xls')
        driver.find_element_by_xpath("//a[@href ='chooseSurvey.aspx']").click()
        time.sleep(3)
    except Exception as ex:
        print(ex)
    
driver.quit()     

C:\Users\kumarsur\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options
